<a href="https://colab.research.google.com/github/tomonari-masada/course2024-nlp/blob/main/02_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# テキスト分類
* BoWでも良い性能を出せることが多い。
  * LLMを使って文書分類するときは、BoW+SVMの性能と比較した方が良い。
  * なぜなら、分類性能に大きな差がつかないことも、しばしばあるので。

## spaCyのインストール

* 最小限のインストール
  * 英語だけ扱えるようになる。

In [ ]:
!conda install -c conda-forge spacy
!python -m spacy download en_core_web_sm

* spaCyで日本語を扱えるようにする。
  * sudachiという形態素解析器が使えるようになる。

In [ ]:
!python -m spacy download ja_core_news_sm

## データセット
* ライブドアニュースコーパスの本文部分を使う。
  * 9値分類。

In [ ]:
from datasets import load_dataset

ds = load_dataset(
    "shunk031/livedoor-news-corpus",
    train_ratio=0.8,
    val_ratio=0.1,
    test_ratio=0.1,
    random_state=42,
    shuffle=True,
    trust_remote_code=True,
)

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['url', 'date', 'title', 'content', 'category'],
        num_rows: 5894
    })
    validation: Dataset({
        features: ['url', 'date', 'title', 'content', 'category'],
        num_rows: 737
    })
    test: Dataset({
        features: ['url', 'date', 'title', 'content', 'category'],
        num_rows: 736
    })
})

In [ ]:
category_names = [
  'movie-enter',
  'it-life-hack',
  'kaden-channel',
  'topic-news',
  'livedoor-homme',
  'peachy',
  'sports-watch',
  'dokujo-tsushin',
  'smax',
]

In [ ]:
ds["train"]["content"][0]

'3日に放送された「サンデージャポン」（TBS系）番組内では、片山さつき議員と元衆議院議員で現在はタレント活動を行う杉村太蔵が、河本準一母の生活保護受給問題について議論し、その内容がネット掲示板やツイッターで話題になっている。  同番組で、河本を擁護する気はなく、制度改革にも賛成すると前置きした杉村だが、「なぜ国会議員が個人攻撃をするのか」と、今回の問題における片山議員の言動を問題視した。すると片山氏は「自分が最初に名前を出したのではない」と反論。しかし、杉村は「最初かどうかに関係なく議員は個人攻撃をするべきではない」と主張し、さらに「片山議員は名前が売りたいだけ」と強く非難した。  このやりとりを受け、ネット掲示板上の反応は「タイゾーも目立ちたいだけだろ」「お前こそ議員時はパフォーマンスだけだったくせに」「サンジャポみたけどタイゾーはただ怒鳴ってるだけだった」と、杉村への批判も多く見られたが、ツイッターでは「片山さつきより余程まともなことを言ってる」「片山さつきを応援してる人は2chが世論だと思ってそう」などと、杉村太蔵の主張を支持するユーザも目立ち、その賛否は五分五分といったところだった。  【関連記事】 ・片山さつき議員\u3000杉村太蔵の批判にも冷静「問題を追及するのが仕事」  【関連情報】 ・杉村太蔵 「あなたは目立ちたいだけだ！個人攻撃は政治手法としておかしい」…生放送で片山議員を激しく批判（痛いニュース）'

## 形態素解析

In [ ]:
import spacy

nlp = spacy.load("ja_core_news_sm")
doc = nlp(ds["train"]["content"][0])
for token in doc:
  print(token.lemma_, end=" ")

3 日 に 放送 する れる た 「 サンデー ジャポン 」 ( TBS 系 ) 番組 内 で は 、 片山 さつき 議員 と 元 衆議 院 議員 で 現在 は タレント 活動 を 行う 杉村 太蔵 が 、 河本 準一 母 の 生活 保護 受給 問題 に つく て 議論 する 、 その 内容 が ネット 掲示 板 や ツイッター で 話題 に なる て いる 。   同 番組 だ 、 河本 を 擁護 する 気 は ない 、 制度 改革 に も 賛成 する と 前置き する た 杉村 だ が 、 「 なぜ 国会 議員 が 個人 攻撃 を する の か 」 と 、 今回 の 問題 に おく り 片山 議員 の 言動 を 問題 視 する た 。 する と 片山 氏 は 「 自分 が 最初 に 名前 を 出す た の だ は ない 」 と 反論 。 しかし 、 杉村 は 「 最初 か どう か に 関係 ない 議員 は 個人 攻撃 を する べし だ は ない 」 と 主張 する 、 さらに 「 片山 議員 は 名前 が 売る たい だけ 」 と 強い 非難 する た 。   この やりとり を 受ける 、 ネット 掲示 板上 の 反応 は 「 タイゾー も 目立つ たい だけ だ 」 「 お前 こそ 議員 時 は パフォーマンス だけ だ た くせ に 」 「 サンジャポ みる た けど タイゾー は ただ 怒鳴る てる だけ だ た 」 と 、 杉村 へ の 批判 も 多い 見る られる た が 、 ツイッター で は 「 片山 さつき より 余程 まとも だ こと を 言う てる 」 「 片山 さつき を 応援 する てる 人 は 2 ch が 世論 だ と 思う て そう 」 など と 、 杉村 太蔵 の 主張 を 支持 する ユーザ も 目立つ 、 その 賛否 は 五 分 五 分 と いう た ところ だ た 。   【 関連 記事 】 ・ 片山 さつき 議員 　 杉村 太蔵 の 批判 に も 冷静 「 問題 を 追及 する の が 仕事 」   【 関連 情報 】 ・ 杉村 太蔵 「 あなた は 目立つ たい だけ だ ! 個人 攻撃 は 政治 手法 と する て おかしい 」 … 生 放送 で 片山 議員 を 激しい 批判 ( 痛い ニュース ) 

In [ ]:
from tqdm.auto import tqdm

corpus_train = []
for text in tqdm(ds["train"]["content"]):
  doc = nlp(text)
  corpus_train.append(" ".join([token.lemma_ for token in doc]))

In [ ]:
with open('livedoor-news-corpus_content_lemmatized.txt', 'w') as f:
  f.write("\n".join(corpus_train) + "\n")

In [ ]:
corpus_train = []
with open('livedoor-news-corpus_content_lemmatized.txt', 'r') as f:
  for text in f:
    corpus_train.append(text)

## TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=10, max_df=0.2)
X = vectorizer.fit_transform(corpus_train)

In [ ]:
X.shape

In [ ]:
corpus_val = []
for text in tqdm(ds["validation"]["content"]):
  doc = nlp(text)
  corpus_val.append(" ".join([token.lemma_ for token in doc]))

corpus_test = []
for text in tqdm(ds["test"]["content"]):
  doc = nlp(text)
  corpus_test.append(" ".join([token.lemma_ for token in doc]))

In [ ]:
import numpy as np

corpus = np.array(corpus_train + corpus_val)
len(corpus)
labels = np.array(ds["train"]["category"] + ds["validation"]["category"])

## ハイパーパラメータのチューニング
* SVMの正則化パラメータ`C`
* TfidfVectorizerの`min_df`と`max_df`

### 1

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

for min_df in [10, 20, 30]:
  for max_df in [0.2, 0.3, 0.4]:
    vectorizer = TfidfVectorizer(min_df=min_df, max_df=max_df)
    for C in 10. ** np.arange(-1, 4):
      scores = []
      skf_split = skf.split(corpus, labels)
      for train_index, val_index in skf_split:
        X_train = vectorizer.fit_transform(corpus[train_index])
        clf = LinearSVC(C=C, dual=False, max_iter=1000, random_state=123)
        clf.fit(X_train, labels[train_index])
        X_val = vectorizer.transform(corpus[val_index])
        score = clf.score(X_val, labels[val_index])
        print(f"\t{score:.3f}", end=" ")
        scores.append(score)
      print(f"\nmean accuracy: {np.array(scores).mean():.3f}", end="")
      print(f" | C={C:.2e} min_df={min_df} max_df={max_df:.3f}")

	0.894 	0.919 	0.909 	0.898 	0.910 
mean accuracy: 0.906 | C=1.00e-01 min_df=10 max_df=0.200
	0.910 	0.931 	0.928 	0.931 	0.928 
mean accuracy: 0.926 | C=1.00e+00 min_df=10 max_df=0.200
	0.911 	0.932 	0.932 	0.932 	0.932 
mean accuracy: 0.928 | C=1.00e+01 min_df=10 max_df=0.200
	0.910 	0.930 	0.931 	0.928 	0.930 
mean accuracy: 0.926 | C=1.00e+02 min_df=10 max_df=0.200
	0.909 	0.928 	0.926 	0.927 	0.928 
mean accuracy: 0.924 | C=1.00e+03 min_df=10 max_df=0.200
	0.895 	0.917 	0.913 	0.904 	0.905 
mean accuracy: 0.907 | C=1.00e-01 min_df=10 max_df=0.300
	0.910 	0.933 	0.936 	0.931 	0.931 
mean accuracy: 0.928 | C=1.00e+00 min_df=10 max_df=0.300
	0.913 	0.933 	0.932 	0.933 	0.934 
mean accuracy: 0.929 | C=1.00e+01 min_df=10 max_df=0.300
	0.909 	0.929 	0.934 	0.930 	0.934 
mean accuracy: 0.927 | C=1.00e+02 min_df=10 max_df=0.300
	0.909 	0.930 	0.929 	0.926 	0.934 
mean accuracy: 0.926 | C=1.00e+03 min_df=10 max_df=0.300
	0.896 	0.922 	0.913 	0.909 	0.905 
mean accuracy: 0.909 | C=1.00e-01 

### 2

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

for min_df in [5, 10, 15]:
  for max_df in [0.3, 0.4, 0.5]:
    vectorizer = TfidfVectorizer(min_df=min_df, max_df=max_df)
    for C in 10. ** np.arange(-1, 4):
      scores = []
      skf_split = skf.split(corpus, labels)
      for train_index, val_index in skf_split:
        X_train = vectorizer.fit_transform(corpus[train_index])
        clf = LinearSVC(C=C, dual=False, max_iter=1000, random_state=123)
        clf.fit(X_train, labels[train_index])
        X_val = vectorizer.transform(corpus[val_index])
        score = clf.score(X_val, labels[val_index])
        print(f"\t{score:.3f}", end=" ")
        scores.append(score)
      print(f"\nmean accuracy: {np.array(scores).mean():.3f}", end="")
      print(f" | C={C:.2e} min_df={min_df} max_df={max_df:.3f}")

	0.894 	0.916 	0.912 	0.901 	0.904 
mean accuracy: 0.906 | C=1.00e-01 min_df=5 max_df=0.300
	0.916 	0.934 	0.931 	0.931 	0.931 
mean accuracy: 0.928 | C=1.00e+00 min_df=5 max_df=0.300
	0.915 	0.936 	0.931 	0.931 	0.932 
mean accuracy: 0.929 | C=1.00e+01 min_df=5 max_df=0.300
	0.913 	0.937 	0.931 	0.932 	0.934 
mean accuracy: 0.929 | C=1.00e+02 min_df=5 max_df=0.300
	0.910 	0.932 	0.928 	0.929 	0.931 
mean accuracy: 0.926 | C=1.00e+03 min_df=5 max_df=0.300
	0.896 	0.915 	0.913 	0.905 	0.908 
mean accuracy: 0.907 | C=1.00e-01 min_df=5 max_df=0.400
	0.914 	0.937 	0.934 	0.931 	0.932 
mean accuracy: 0.930 | C=1.00e+00 min_df=5 max_df=0.400
	0.920 	0.937 	0.934 	0.934 	0.937 
mean accuracy: 0.932 | C=1.00e+01 min_df=5 max_df=0.400
	0.921 	0.934 	0.931 	0.934 	0.937 
mean accuracy: 0.931 | C=1.00e+02 min_df=5 max_df=0.400
	0.916 	0.934 	0.931 	0.931 	0.935 
mean accuracy: 0.929 | C=1.00e+03 min_df=5 max_df=0.400
	0.903 	0.919 	0.918 	0.905 	0.907 
mean accuracy: 0.910 | C=1.00e-01 min_df=5 m

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

for min_df in [15, 20, 30]:
  for max_df in [0.5, 0.7, 1.0]:
    vectorizer = TfidfVectorizer(min_df=min_df, max_df=max_df)
    for C in [10.0]:
      scores = []
      skf_split = skf.split(corpus, labels)
      for train_index, val_index in skf_split:
        X_train = vectorizer.fit_transform(corpus[train_index])
        clf = LinearSVC(C=C, dual=False, max_iter=1000, random_state=123)
        clf.fit(X_train, labels[train_index])
        X_val = vectorizer.transform(corpus[val_index])
        score = clf.score(X_val, labels[val_index])
        print(f"\t{score:.3f}", end=" ")
        scores.append(score)
      print(f"\nmean accuracy: {np.array(scores).mean():.3f}", end="")
      print(f" | C={C:.2e} min_df={min_df} max_df={max_df:.3f}")

	0.924 	0.939 	0.939 	0.937 	0.935 
mean accuracy: 0.935 | C=1.00e+01 min_df=15 max_df=0.500
	0.929 	0.940 	0.941 	0.942 	0.943 
mean accuracy: 0.939 | C=1.00e+01 min_df=15 max_df=0.700
	0.934 	0.940 	0.943 	0.943 	0.941 
mean accuracy: 0.940 | C=1.00e+01 min_df=15 max_df=1.000
	0.924 	0.937 	0.933 	0.932 	0.936 
mean accuracy: 0.932 | C=1.00e+01 min_df=20 max_df=0.500
	0.928 	0.938 	0.937 	0.941 	0.941 
mean accuracy: 0.937 | C=1.00e+01 min_df=20 max_df=0.700
	0.929 	0.943 	0.939 	0.944 	0.943 
mean accuracy: 0.940 | C=1.00e+01 min_df=20 max_df=1.000
	0.923 	0.935 	0.925 	0.934 	0.936 
mean accuracy: 0.931 | C=1.00e+01 min_df=30 max_df=0.500
	0.927 	0.940 	0.932 	0.942 	0.943 
mean accuracy: 0.937 | C=1.00e+01 min_df=30 max_df=0.700
	0.924 	0.942 	0.934 	0.944 	0.937 
mean accuracy: 0.936 | C=1.00e+01 min_df=30 max_df=1.000


In [ ]:
vectorizer = TfidfVectorizer(min_df=20)
X = vectorizer.fit_transform(corpus)
clf = LinearSVC(C=10.0, dual=False, max_iter=1000, random_state=123)
clf.fit(X, labels)
X_test = vectorizer.transform(corpus_test)
score = clf.score(X_test, ds["test"]["category"])
print(f"{score:.3f}")


0.947


# 課題
* この分類性能を改良できるかどうか、試行錯誤してみてください。
  * 注意：データセットの分割の仕方は変えないように。